In [27]:
import pandas as pd
import numpy as np
import nltk

In [28]:
df = pd.read_excel("Data-FakeRealCOVID.xlsx")

In [29]:
df

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
...,...,...,...
6415,6416,A tiger tested positive for COVID-19 please st...,fake
6416,6417,???Autopsies prove that COVID-19 is??� a blood...,fake
6417,6418,_A post claims a COVID-19 vaccine has already ...,fake
6418,6419,Aamir Khan Donate 250 Cr. In PM Relief Cares Fund,fake


In [30]:
# df.loc[i,'tweet'].lower()
# df.tweet=df.tweet.apply(lambda x : x.lower())
df.tweet=df.tweet.str.lower()

# df.loc[0,'tweet'].lower()

In [31]:
df

,id,tweet,label
0,1,the cdc currently reports 99031 deaths. in gen...,real
1,2,states reported 1121 deaths a small rise from ...,real
2,3,politically correct woman (almost) uses pandem...,fake
3,4,#indiafightscorona: we have 1524 #covid testin...,real
4,5,populous states can generate large case counts...,real
...,...,...,...
6415,6416,a tiger tested positive for covid-19 please st...,fake
6416,6417,???autopsies prove that covid-19 is??� a blood...,fake
6417,6418,_a post claims a covid-19 vaccine has already ...,fake
6418,6419,aamir khan donate 250 cr. in pm relief cares fund,fake


In [34]:
# import re
# df.loc[1,'tweet'] = re.sub(r'^https?:\/\/.*[\r\n]*', '', df.loc[1,'tweet'] , flags=re.MULTILINE)

import re 
for i in range(df.shape[0]):
    df.loc[i,'tweet']=re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", df.loc[i,'tweet'])


In [36]:
df.loc[4,'tweet']

'populous states can generate large case counts but if you look at the new cases per million today 9 smaller states are showing more cases per million than california or texas: al ar id ks ky la ms nv and sc.  '